In [1]:
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV

In [2]:
file_path = "all_w10_s1.arff"
data, meta = arff.loadarff(file_path)
df = pd.DataFrame(data)

In [3]:
def convert_binary_strings(df):
    for column in df.columns:
        if df[column].dtype == object:
            df[column] = df[column].str.decode('utf-8')
    return df
df = convert_binary_strings(df)

In [4]:
X = df.iloc[:, :-1].values  
y = df.iloc[:, -1].values   
num_columns = X.shape[0] 
print(num_columns)
#missing:
print("missing values：")
print(pd.DataFrame(X).isnull().sum())
#train and test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

27588
missing values：
0      0
1      0
2      0
3      0
4      0
      ..
183    0
184    0
185    0
186    0
187    0
Length: 188, dtype: int64


In [8]:
#step 1:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 训练和评估模型的函数
def evaluate_model(X_train, X_test, y_train, y_test, feature_extraction_method):
    steps = [('scaler', StandardScaler())]
    print('*'*50)
    if feature_extraction_method == 'poly':
        print("Choosing Polynomial for feature extraction!")
        steps.append(('poly_features', PolynomialFeatures(degree=1)))  
    elif feature_extraction_method == 'lda':
        print("Choosing LDA for feature extraction!")
        steps.append(('pca', PCA(n_components=10))) 
        steps.append(('lda', LinearDiscriminantAnalysis()))  
    elif feature_extraction_method == 'pca':
        print("Choosing PCA for feature extraction!")
        steps.append(('pca', PCA(n_components=50)))  
    
    #classfier:RF
    steps.append(('classifier', RandomForestClassifier(n_estimators=50)))
    pipeline = Pipeline(steps)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

methods = ['poly', 'lda', 'pca']
results = {}

for method in methods:
    accuracy = evaluate_model(X_train, X_test, y_train, y_test, method)
    results[method] = accuracy
    print(f"{method} accuracy: {accuracy}")
    print('*'*50)

# 对比结果
print("\nComparison of feature extraction methods:")
for method, accuracy in results.items():
    print('*'*50)
    print(f"{method}: {accuracy}")
    print('*'*50)

**************************************************
Choosing Polynomial for feature extraction!
poly accuracy: 0.9983085659055213
**************************************************
**************************************************
Choosing LDA for feature extraction!
lda accuracy: 0.8617856711368853
**************************************************
**************************************************
Choosing PCA for feature extraction!
pca accuracy: 0.9492569771656397
**************************************************

Comparison of feature extraction methods:
**************************************************
poly: 0.9983085659055213
**************************************************
**************************************************
lda: 0.8617856711368853
**************************************************
**************************************************
pca: 0.9492569771656397
**************************************************


In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings("ignore")
#step 2:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#model function:
def evaluate_model(X_train, X_test, y_train, y_test, model_name):
    if model_name == 'Perceptron':
        model = Perceptron()
    elif model_name == 'Logistic Regression':
        model = LogisticRegression()
    elif model_name == 'SVM':
        model = SVC()
    elif model_name == 'Decision Tree':
        model = DecisionTreeClassifier()
    elif model_name == 'Random Forest':
        model = RandomForestClassifier(n_estimators=50)
    elif model_name == 'AdaBoost':
        model = AdaBoostClassifier(n_estimators=50)
    elif model_name == 'MLP':
        model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=100)
    #pipline:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model)
    ])
    
    #train
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1

#model choose
models = ['Perceptron', 'Logistic Regression', 'SVM', 'Decision Tree', 'Random Forest', 'AdaBoost', 'MLP']
results = {}

for model_name in models:
    accuracy, precision, recall, f1 = evaluate_model(X_train, X_test, y_train, y_test, model_name)
    results[model_name] = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1}
    print(f"{model_name}:\n Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}\n")

#print reslut:
print("\nModel Performance Comparison:")
for model_name, metrics in results.items():
    print(f"{model_name}: {metrics}")


Perceptron:
 Accuracy: 0.35212033345415006, Precision: 0.3918154538115788, Recall: 0.35212033345415006, F1 Score: 0.3331010931673871

Logistic Regression:
 Accuracy: 0.7180137731061979, Precision: 0.7297433014105574, Recall: 0.7180137731061979, F1 Score: 0.6975894447443177

SVM:
 Accuracy: 0.43530264588619066, Precision: 0.46214620616047725, Recall: 0.43530264588619066, F1 Score: 0.42460305005938687

Decision Tree:
 Accuracy: 0.8218557448350852, Precision: 0.8429282106584832, Recall: 0.8218557448350852, F1 Score: 0.8192707826160065

Random Forest:
 Accuracy: 0.948169626676332, Precision: 0.9545308432328902, Recall: 0.948169626676332, F1 Score: 0.9481064684248458

AdaBoost:
 Accuracy: 0.003624501631025734, Precision: 0.0005954457849877821, Recall: 0.003624501631025734, F1 Score: 0.0008694557165192725

MLP:
 Accuracy: 0.8251177963030083, Precision: 0.8313444025100708, Recall: 0.8251177963030083, F1 Score: 0.8103996304196759


Model Performance Comparison:
Perceptron: {'Accuracy': 0.35212

In [ ]:
#step 3:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#pipline:
steps = [
    ('scaler', StandardScaler()),  
    ('pca', PCA(n_components=50)),
    ('classifier', RandomForestClassifier(random_state=42))  
]

pipeline = Pipeline(steps)
#rf metric
param_grid = {
    'classifier__n_estimators': [50, 100, 200],  
    'classifier__max_depth': [None, 5, 10],  
    #'classifier__min_samples_split': [2, 5, 10],  
    'classifier__min_samples_leaf': [1, 2],  
}
#GridSearchCV Search
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)
print("Best parameters found: ", grid_search.best_params_)
y_pred = grid_search.predict(X_test)


Fitting 5 folds for each of 18 candidates, totalling 90 fits
Best parameters found:  {'classifier__max_depth': None, 'classifier__min_samples_leaf': 1, 'classifier__n_estimators': 200}
Classification Result:


In [17]:
print("Classification Result:")
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("best model accuracy:",accuracy)
print("best model precision:",precision)
print("best model recall:",recall)
print("best model f1:",f1)

Classification Result:
best model accuracy: 0.9516733115863235
best model precision: 0.9561342910460174
best model recall: 0.9516733115863235
best model f1: 0.9514655207144608
